In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab 
import nltk

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.moses import MosesDetokenizer

#read csv into dataframe
#GOPSentiDf = pd.read_csv('2016GOPPresDebSenti.csv')
AirlineSentiDf = pd.read_csv('TweetsUSAirlineSenti.csv')

#Begin feature removal
#GOPSentiDf.count()

In [2]:
AirlineSentiDf.count()

tweet_id                        14640
airline_sentiment               14640
airline_sentiment_confidence    14640
negativereason                   9178
negativereason_confidence       10522
airline                         14640
airline_sentiment_gold             40
name                            14640
negativereason_gold                32
retweet_count                   14640
text                            14640
tweet_coord                      1019
tweet_created                   14640
tweet_location                   9907
user_timezone                    9820
dtype: int64

In [3]:
#we want the sentiment analysis to be as general as possible, independent of user location, when they tweeted, 
#the tweet's subject matter, how often their tweets get retweeted, who is the user, 

'''GOPSentiDf.drop('user_timezone', 1, inplace = True)
GOPSentiDf.drop('tweet_location', 1, inplace = True)
GOPSentiDf.drop('tweet_id', 1, inplace = True)
GOPSentiDf.drop('tweet_created', 1, inplace = True)
GOPSentiDf.drop('tweet_coord', 1, inplace = True)
#subject matter gold is the specific topic of tweeted text, such as Religion, Abortion, Immigration, FOX news, etc.
GOPSentiDf.drop('subject_matter_gold', 1, inplace = True)
#sentiment gold is repetitive of the sentiment column
GOPSentiDf.drop('sentiment_gold', 1, inplace = True)
GOPSentiDf.drop('retweet_count', 1, inplace = True)
#relevant_yn_gold is repetitive of relevant
GOPSentiDf.drop('relevant_yn_gold', 1, inplace = True)
GOPSentiDf.drop('name', 1, inplace = True)
#candidate_gold is repetitive of candidate
GOPSentiDf.drop('candidate_gold', 1, inplace = True)
GOPSentiDf.drop('subject_matter_confidence', 1, inplace = True)
GOPSentiDf.drop('subject_matter', 1, inplace = True)
GOPSentiDf.drop('relevant_yn_confidence', 1, inplace = True)
GOPSentiDf.drop('relevant_yn', 1, inplace = True)
GOPSentiDf.drop('candidate_confidence', 1, inplace = True)
GOPSentiDf.drop('candidate', 1, inplace = True)'''

#we want to do the same for the tweets for the US Airlines
AirlineSentiDf.drop('user_timezone', 1, inplace = True)
AirlineSentiDf.drop('tweet_location', 1, inplace = True)
AirlineSentiDf.drop('tweet_created', 1, inplace = True)
AirlineSentiDf.drop('tweet_coord', 1, inplace = True)
AirlineSentiDf.drop('retweet_count', 1, inplace = True)
AirlineSentiDf.drop('negativereason_gold', 1, inplace = True)
AirlineSentiDf.drop('name', 1, inplace = True)
AirlineSentiDf.drop('airline_sentiment_gold', 1, inplace = True)
AirlineSentiDf.drop('airline', 1, inplace = True)
AirlineSentiDf.drop('negativereason_confidence', 1, inplace = True)
AirlineSentiDf.drop('negativereason', 1, inplace = True)

print("removed unnecessary columns")

removed unnecessary columns


In [4]:
#GOPSentiDf.count()

In [5]:
AirlineSentiDf.count()

tweet_id                        14640
airline_sentiment               14640
airline_sentiment_confidence    14640
text                            14640
dtype: int64

In [6]:
#list of emotions, not emojis
#create 2d array with 50 columns and 3 rows, where the row determines the number of character per emoji
#i.e., :) contains 2 characters so it belongs it row 0, while :-D goes to row 1, and so on
PosEmoAr = [[':)', ':]', ':}', '=)', '=]', '=}', ':B', '=B', '<3', '^^', ':*', '=*', ';)', ';]', 
             ';}', '=P', '=p', ':P', ':p', ':b', '=b'], 
            [':o)', ':o]', ':o}', ':-]', ':-)', ':-}', '=^]', '=^)', '=^}', ':-D', ':-B', 
             ':^D', ':^B', '=^B', '=^D', ':\')', ':\']', '=\'}', '^.^', '^-^', '^_^', ':-*', 
             ':-p', ':-P', ':-b', ':^p', ':^P', ':^b', '\\o\\', '/o/', '=^p', '=^P', '=^b', '\\o/']]
NegEmoAr = [['D:', 'D=', ':(', ':[', ':{', '=(', '=[', '={', '=\\', ':\\', '=/', ':/', '=$', 'Oo'], 
            ['D-:', 'D^:', 'D^=', ':o(', ':o[', ':^(', ':^[', ':^{', '=^(', '=^{', '>=(', '>=[', '>={', 
             ':-[', ':-(', '=^[', '>=[', ':\'(', ':\'[', ':\'{', '=\'{', '=\'(', '=\'[', 'o.O', 'O_o', ':o{'],
            ['>:-{', '>:-[', '>:-(', '>=^[', '>=^(', '>:-{', '>=^{'],
            ['>:-=(', ':$:-{']]
NeuEmoAr = [[':|', '=|', '><', ':o', ':O', '=0', ':@', '=@', ':x', '=X', ':#', '=#'], 
            [':-|', '>.<', '>_<', ':^o', ':^@', '-.-', '-_-', ':-x', ':-X', ':-@', ':-#', ':^x', ':^#'], 
            ['-.-\'', '-_-\'']]

print("created emoticons comparator")

created emoticons comparator


In [20]:
#create features: number of negative and positive words and emoticons, number of emoticons
'''GOPTxtAr = GOPSentiDf.values[0:,][:,3]
GOPNumPosAr = []
GOPNumNegAr = []
GOPNumEmoAr = []
for i in range(len(GOPTxtAr)):
    GOPNumPosAr.append(0)
    GOPNumNegAr.append(0)
    GOPNumEmoAr.append(0)'''
    
AirTxtAr = AirlineSentiDf.values[0:,][:,3]
AirNumPosAr = []
AirNumNegAr = []
AirNumEmoAr = []

for i in range(len(AirTxtAr)):
    AirNumPosAr.append(0)
    AirNumNegAr.append(0)
    AirNumEmoAr.append(0)

print("created new features")

created new features


In [21]:
#remove all stopwords, hashtags, web links, retweets (RT), and direct @s
stop_words = set(stopwords.words("english"))

'''index = 0
shrtTxtGOPAr = []
for i in range(len(GOPTxtAr)):
    shrtTxtGOPAr.append('')
for text in GOPTxtAr:
    GOPTxtAr[index] = GOPTxtAr[index].replace('RT ', '')
    while (GOPTxtAr[index].find('#') != -1):
        GOPTxtAr[index] = GOPTxtAr[index].replace(GOPTxtAr[index][GOPTxtAr[index].find('#') : GOPTxtAr[index].find(' ', GOPTxtAr[index].find('#'))], '')
    while (GOPTxtAr[index].find('@') != -1):
        GOPTxtAr[index] = GOPTxtAr[index].replace(GOPTxtAr[index][GOPTxtAr[index].find('@') : GOPTxtAr[index].find(' ', GOPTxtAr[index].find('@'))], '')
    while (GOPTxtAr[index].find('http') != -1):
        GOPTxtAr[index] = GOPTxtAr[index].replace(GOPTxtAr[index][GOPTxtAr[index].find('http') : ], '')
    text = GOPTxtAr[index]
    GOPTxtAr[index] = ''
    words = word_tokenize(text)
    for w in words:
        if w not in stop_words:
            shrtTxtGOPAr[index] = shrtTxtGOPAr[index] + ' ' + w
    GOPTxtAr[index] = shrtTxtGOPAr[index]
    index += 1'''

index = 0
shrtTxtAirAr = []
for i in range(len(AirTxtAr)):
    shrtTxtAirAr.append('')
for text in AirTxtAr:
    AirTxtAr[index] = AirTxtAr[index].replace('RT ', '')
    while (AirTxtAr[index].find('#') != -1):
        AirTxtAr[index] = AirTxtAr[index].replace('#', '')
    while (AirTxtAr[index].find('@') != -1):
        AirTxtAr[index] = AirTxtAr[index].replace('@', '')
    while (AirTxtAr[index].find('http') != -1):
        cnt = AirTxtAr[index].find('http')
        AirTxtAr[index] = AirTxtAr[index].replace(AirTxtAr[index][cnt : ], '')
    text = AirTxtAr[index]
    AirTxtAr[index] = ''
    words = word_tokenize(text)
    for w in words:
        if w not in stop_words:
            shrtTxtAirAr[index] = shrtTxtAirAr[index] + ' ' + w
    AirTxtAr[index] = shrtTxtAirAr[index]
    print(AirTxtAr[index])
    index += 1

 VirginAmerica What dhepburn said .
 VirginAmerica plus 've added commercials experience ... tacky .
 VirginAmerica I n't today ... Must mean I need take another trip !
 VirginAmerica 's really aggressive blast obnoxious `` entertainment '' guests ' faces & amp ; little recourse
 VirginAmerica 's really big bad thing
 VirginAmerica seriously would pay $ 30 flight seats n't playing . 's really bad thing flying VA
 VirginAmerica yes , nearly every time I fly VX “ ear worm ” ’ go away : )
 VirginAmerica Really missed prime opportunity Men Without Hats parody , .
 virginamerica Well , I didn't…but NOW I DO ! : -D
 VirginAmerica amazing , arrived hour early . You 're good .
 VirginAmerica know suicide second leading cause death among teens 10-24
 VirginAmerica I & lt ; 3 pretty graphics . much better minimal iconography . : D
 VirginAmerica This great deal ! Already thinking 2nd trip Australia & amp ; I n't even gone 1st trip yet ! ; p
 VirginAmerica virginmedia I 'm flying fabulous Seducti

 VirginAmerica Late Flight bag check lost business . I missed flight AM apt . Three people flight exp .
 VirginAmerica - amazing customer service , ! 💕💕 RaeAnn SF - 's best ! customerservice virginamerica flying
 VirginAmerica called service line hung . This awesome . sarcasm
 VirginAmerica site tripping . I 'm trying check I 'm getting plain text version . I reluctant enter card info .
 VirginAmerica I scheduled SFO 2 DAL flight 714 today . Changed 24th due weather . Looks like flight still ?
 VirginAmerica getaway deals May , $ 59 one-way . Lots cool cities
 VirginAmerica getaway deals May , $ 59 one-way . Lots cool cities
 VirginAmerica getaway deals May , $ 59 one-way . Lots cool cities
 VirginAmerica getaway deals May , $ 59 one-way . Lots cool cities
 VirginAmerica Have great week 🌞✈
 VirginAmerica come back PHL already . We need take us horrible cold . pleasecomeback
 VirginAmerica I concerned I fly plane needs delayed due `` tech stop '' ?
 VirginAmerica best airline I flown on

 united That 's correct—I 've spent hours trying book online receive error clicking final purchase button . ( 1/2 )
 united Today finest . All could prevented one gate agent advising 200 passengers .
 united real deal delay flight UA 1032 Denver Vegas ?
 united UA63 Madrid arrived agent mass confusion sent fro 3 UA employees . MaryJo rude unhelpful .
 united system failure = bad trend . Software projects like flying airplane ; 's thing emergency takeoff .
 united lots reports system failures delaying flights last week . Currently sitting tarmac OGG hour .
 united rebooting Chicago dispatch system , need pivotalcf I 'm tired sitting planes
 united despite shaky connections , looks like I 'll get home tonight . Great job , united - touch go while..
 united
 united aircraft closest gate flight one closest gate going Chicago .
 united tell file claim clothing purchased ?
 united comically return flight ASE two United flights boarding adjacent gates clear plane board
 united GRK13575M file 

 united 's boarding time ua3882 crewe
 united added one . Ty
 united thanks effort . I get earliest . Though I make complaint refund I losing one business day tomorrow
 united . Pls hold UA2066 9 cnx pax frm UA6194 . All runners . A 5 min hold save 2 fam frm miscnx . Pls . Twitter watching .
 united way one person working entire security checkpoint EWR 🆖
 united THANK YOU !
 united authors fiction ? I heard radio n't even jetway secured yet UA5037 CMH
 united ? Found anything showing ? You realise morning flight Cancelled Flightled pricey evening ?
 united oh I . Was still lost . Your tracking system stinks delivery system .
 united feeling like true united customer stuck.on tarmac dc 2 hrs . UNITEDFAIL
 united pls follow DM .
 united leadership counts . landed 11:15 last night-deplaned 12:40 . 45 minutes waiting jetway . Inexcusable
 united I sent DM requested heard anything . Can address issue I brought DM explain happened ?
 united employee=faceless . Do n't throw bus . Starts manag

 united On plane thanks ! Maybe n't let many people check - teeming morons .
 united Thanks ... seat made 6 year old broken TV . 20 flights/year ... last airline .
 united UA6357 needs de-icing ! Stat ! We 've waited turn ...
 united yes I 've boarded way many times & amp ; never show pass Tarmac multiple times . Path railed . Only 1 way
 united bobwesson fair enough United . everybody best . Although `` slight delay '' turning Cancelled Flightlation .
 united Still waiting bag ! Never got delivered yesterday . Ca n't reach real person help . Sending DM ref number .
 united worst flight experience I 've ever . Will never ever fly airline .
 united I go thru details flight I want w/ recording repeat ALL real person gets line ?
 united Looks like came . Thanks help .
 united anything yet JJ ?
 united would person take hours hours wait time tell lie
 united hey ! DM working . UA 307 !
 united refund team far customer care courtesy need training work ethics . So 's like sir u r lying I hel

...

 USAirways really ? 8 hr delay Virgin Island even food voucher .
 usairways . Worst . Ever . dca customerservice
 USAirways 2nd February row- stranded us try return Cancun ! No contact info- way rebook ! badcustomerservice
 USAirways printer problems causing us sit runway 45 mins . Get together .
 USAirways nothing like back back delayed flights let hope one n't 12 hours like last time .
 USAirways please help find luggage since 're one lost ? Your customer service SUCKS .
 USAirways What happening Reagan Airport DCA ? Long lines four people working checkIn ! unhappy DCA
 USAirways AuroraBIZ check bag , PHLAirport bag handlers broke suitcase stole camera , bag returned trash bag
 usairways hopefully crappy service improves u become americanairlines angryairtravel winterstorm
 usairways 2 people working counter dca ? Massive lines . Poorly played . winterstorm
 USAirways 's going CLT ?
 USAirways USAirwaysCenter My lost bag lands Hartford 639p tonight . Will delivered Danbury CT tonight

 USAirways policy around bringing breast pump board ?
 USAirways Thanks . No DC yet , I see . I keep link check back . Appreciate !
 USAirways I 'm glad . But I spend hour try track basic information already available .
 USAirways Will guys offering no-fee changes flights DCA tomorrow afternoon ?
 USAirways get ahold reservations supervisor ?
 USAirways I called BTV desk ; information - exists . Your systems n't make easy customer access .
 USAirways awesome ! And yes , UpInAirClaire !
 USAirways website give correct info said bag Miami key west entire time..I got run around 2 days
 USAirways hold 45 minutes speaking 2 lady airport counter . Gave
 USAirways I want find bag I wo n't go hassle trying claim suitcase usairwaysfail
 USAirways Literally says `` Case open '' What good ?
 USAirways sorry . I trying 2 add day 2 vacation $ 200 charge 2 change flights
 USAirways Melinda reservations greensboro nc absolute jewel thank 's amazing ! ! !
 USAirways two women workng PHL bag check term

 AmericanAir sorry Late Flight , responded DM .
 AmericanAir Believe , I understand . Flight 2955 . Was originally booked Sunday . Flight Cancelled Flighted rescheduled today .
 AmericanAir aa employees rude unwilling help . 10,000 miles rotten cherry top dog shit Sunday . nocareforcustomers
 AmericanAir Mold flight ? ! ? US3825 filthyplane hopeidonotgetsick
 AmericanAir 767 seconds touchdown Madrid airport April 2013 AvGeek
 AmericanAir I slept miami airport due mechanical issues given 10,000 bonus miles try make right . slapintheface
 AmericanAir new 9:45 time confirmed may get Cancelled Flightled ? Traveling kids , need certain . Thx
 AmericanAir 1hr 46 min . Cost flight change $ 788 . Was $ 188 2hrs ago b/f drop call . Cancelled Flighted flight . Asked 4 refund .
 AmericanAir 's frustrating -- PAID ! get refund ?
 AmericanAir DM locator code , thanks .
 AmericanAir thank !
 AmericanAir I sure hope fix USAirways . Good luck , service sucks . nexttimeiwillflysouthwest
 americanair th

 AmericanAir Thanks AA upgrade today getting new flight first one Cancelled Flightled !
 AmericanAir worst flight change policy . No mercy , sympathy , bummer ca n't go funerals see friends bc .
 AmericanAir 's done done . Not much point . Trip already .
 AmericanAir basically u right
 AmericanAir epicfail connections Chicago today , extremely disappointed w/ unaccommodating customer service , rethinking loyalty 😐
 AmericanAir well I 'm flying JetBlue instead - free ! Their credits actually work .
 AmericanAir messaged . Please someone contact us immediately .
 AmericanAir ca n't dfwairport capability spread salt runway ? !
 AmericanAir 3349 , supposed little rock 10am morning .
 AmericanAir boom .
 AmericanAir already spoke line , unwilling help - really poor support hitawall
 AmericanAir first ride new 737-800 new interior seat video . Nice improvement ! Newplanesmell
 AmericanAir No worries . Y ’ good one ! !
 AmericanAir Yes , thanks I found , n't see gray tab first : )
 AmericanAi

 AmericanAir 're AA1401 landed 8:55pm Miami , waiting gate . Can someone team help us ? !
 AmericanAir - bags ? Baggage claim C25 C26 ( flt 959 ) ? ? I see plane ? Why take 25 min ? ? ?
 AmericanAir one call centre ? I know 's busy right , really need speak agent computer
 AmericanAir guys fail again.. I need use voucher ticket I ca n't online yourphonesystemsucks
 AmericanAir Shout-out ground crews working cold ! Thanks y'all . ( & amp ; social media team customer service )
 AmericanAir I fortunately flight - waiting area / impressed w handled stressful situation
 AmericanAir 've hold 4 hrs , Cancelled Flighted flt 2222 phl-dfw . Need assistance ! ! !
 AmericanAir thanks Marie reFlight Booking Problems friends ! ! ! I 'm Cali bound thanks sweet angel ! blessed
 AmericanAir flights Buffalo well -- You flight Cancelled Flightled ! ! !
 AmericanAir -- This Cancelled Flightled flight Buffalo `` weather '' lied us said airlines Cancelled Flightled
 AmericanAir know I could use flight Canad